# Agentic workflow with Agent Engine A2A integration example

This notebook shows an example of an agentic workflow using remote A2A agents deployed on Agent Engine.

See the following notebook for the workflow description.

- [Multi-agent workflow example](https://github.com/google-cloud-japan/sa-ml-workshop/blob/main/blog/multi_agent_article_writing_example_en.ipynb)

Before running the codes below, create a VertexAI workbench instance by following the **Create a Vertex AI workbench instance** section of the notebook above.

## Preparation

In [ ]:
%pip install --user \
    google-adk==1.14.1 \
    google-genai==1.36.0 \
    google-cloud-aiplatform==1.113.0 \
    a2a-sdk==0.3.5

In [13]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

In [1]:
import httpx, json, os, uuid
from google.auth import default
from google.auth.transport.requests import Request
import vertexai

# ADK
from google.genai.types import Part, Content, HttpOptions
from google.adk.agents.callback_context import CallbackContext
from google.adk.agents.llm_agent import LlmAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.agents.sequential_agent import SequentialAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.auth.credential_service.in_memory_credential_service import InMemoryCredentialService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.models import LlmResponse, LlmRequest
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, VertexAiSessionService

# A2A
from a2a.client import ClientConfig, ClientFactory
from a2a.types import AgentCard, TransportProtocol
from google.adk.a2a.executor.a2a_agent_executor import A2aAgentExecutor
from google.adk.a2a.utils.agent_card_builder import AgentCardBuilder

# Agent Engine
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import A2aAgent
from vertexai.preview.reasoning_engines.templates.a2a import create_agent_card


[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
[PROJECT_NUMBER] = !gcloud projects describe {PROJECT_ID} --format='value(projectNumber)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION,
              staging_bucket=f'gs://{PROJECT_ID}')

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

Local application class to execute the root agent.

In [2]:
class LocalApp:
    def __init__(self, agent, app_name='default_app', user_id='default_user'):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = None
        
    async def stream(self, query):
        if not self._session:
            self._session = await self._runner.session_service.create_session(
                app_name=self._app_name,
                user_id=self._user_id,
                session_id=uuid.uuid4().hex,
            )
        content = Content(role='user', parts=[Part(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

## Define agents to be deployed as remote A2A agents

In [3]:
instruction = '''
Your role is to gather the necessary information for writing an article and compile it into a research report.
You will create a list of about 5 topics to be used as a reference when writing an article on a specified theme.
A subsequent agent will create the research report based on this list.

* Output Format
Output in English.
'''

research_agent1 = LlmAgent(
    name='research_agent1',
    model='gemini-2.5-flash',
    description='''
An agent that gathers the necessary information for writing an article
and compiles it into a report (theme selection)
    ''',
    instruction=instruction,
)

In [4]:
instruction = '''
Your role is to collect the necessary information to write an article and compile it into a research report.
The preceding agent will specify about five research topics.

* Output format
Output in Enlgish.
The research report should summarize objective information for each topic.
Each topic must be described in five sentences or more.
'''

research_agent2 = LlmAgent(
    name='research_agent2',
    model='gemini-2.5-flash',
    description='''
An agent that collects the information necessary for writing an article
and compiles it into a report (report creation)
    ''',
    instruction=instruction,
)

In [5]:
instruction = '''
Your role is to write a light, feature article on a specific topic.
You will be given the article's "theme" and a "research report" related to its content.
Based on the objective facts described in the research report, please write a reliable feature article.

If a reviewer points out corrections, please rewrite the article you just wrote according to the feedback.

* Output Conditions
- Output in Enlgish.
- The content should be something that can be casually read in a few minutes, assuming the reader has some basic knowledge of the topic.
- Use a relatively casual and conversational tone.
- Do not include an article title; use the following structure. Arrange each section title to fit the content.
  0. Introduction: Without a section title, write a one- or two-sentence introduction that will make someone want to read the article.
  1. Overview: Summarize and briefly explain the overall picture of the topic.
  2. Latest Information: Feature new information that is particularly noteworthy.
  3. Practical Application: Introduce one thing related to the topic that readers might want to try for themselves.
  4. Conclusion

- Use markdown ## headers for each section title. Use markdown for bullet points as needed.
- Do not use any other markdown formatting.
'''

writer_agent = LlmAgent(
    name='writer_agent',
    model='gemini-2.5-flash',
    description='An agent that writes feature articles on specific topics.',
    instruction=instruction,
)

In [6]:
instruction = f'''
Your role is to review a feature article and provide comments for improvement to ensure its content meets the article's requirements.

* Article Requirements
- The article must begin with a title of less than about 10 words.
  The title should contain useful life information that is applicable from today, making the reader feel they "must read it now."
  The title must use a markdown # header.
- Immediately after the title, add an introduction that summarizes "why this topic is being addressed now" to motivate the reader.
- Use emojis in each section's subtitle to create a friendly feel.
- It must introduce at least three concrete examples that the reader can put into practice starting today.

* Output Format
- Output in English.
- First, explain the good points of the article.
- Next, list the points for correction in a bulleted format.
'''

review_agent = LlmAgent(
    name='review_agent',
    model='gemini-2.5-flash',
    description='An agent that reviews feature articles.',
    instruction=instruction,
)

## Deploy agents on Agent Engine

In [7]:
# Wrapper class of VertexAiSessionService that's compatible with InMemorySessionService
class MyVertexAiSessionService(VertexAiSessionService):

    async def create_session(self, app_name, user_id, state={}, session_id=None):
        # Ignore session_id
        session = await super().create_session(
            app_name=app_name,
            user_id=user_id,
            state=state,
        )
        return session

    async def get_session(self, app_name, user_id, session_id):
        # Return None if session non exists.
        try:
            session = await super().get_session(
                app_name=app_name,
                user_id=user_id,
                session_id=session_id,
            )
            return session
        except:
            return None


def get_create_runner_class(agent, resource_name):
    async def create_runner():
        return Runner(
            # Use resource_name as app_name for VertexAiSessionService.
            app_name=resource_name or agent.name,
            agent=agent,
            artifact_service=InMemoryArtifactService(),
            session_service=MyVertexAiSessionService(),
            memory_service=InMemoryMemoryService(),
            credential_service=InMemoryCredentialService(),
        )
    return create_runner


def get_agent_executor_class(agent, resource_name):
    def agent_executor_builder():
        return A2aAgentExecutor(
            runner=get_create_runner_class(agent, resource_name),
        )       
    return agent_executor_builder


async def get_agent_card(agent):
    builder = AgentCardBuilder(agent=agent)
    adk_agent_card = await builder.build()
    return create_agent_card(
        agent_name=adk_agent_card.name,
        description=adk_agent_card.description,
        skills=adk_agent_card.skills
    )

In [8]:
def get_agent_resource(agent_name):
    for agent in agent_engines.list():
        if agent.display_name == agent_name:
            return agent.resource_name
    return None

In [11]:
agents = {
    'research_agent1_a2a': research_agent1,
    'research_agent2_a2a': research_agent2,
    'writer_agent_a2a': writer_agent,
    'review_agent_a2a': review_agent,
}

client = vertexai.Client(
    project=PROJECT_ID,
    location=LOCATION,
    http_options=HttpOptions(
        api_version='v1beta1', base_url=f'https://{LOCATION}-aiplatform.googleapis.com/'
    ),
)

for agent_name, agent in agents.items():    
    a2a_agent = A2aAgent(
        agent_card=await get_agent_card(agent),
        agent_executor_builder=get_agent_executor_class(agent, None)
    )
    config={
        'display_name': agent_name,
        'description': a2a_agent.agent_card.description,
        'requirements': [
            'google-adk==1.14.1',
            'google-genai==1.36.0',
            'google-cloud-aiplatform==1.113.0',
            'a2a-sdk==0.3.5'
        ],
        'http_options': {
            'base_url': f'https://{LOCATION}-aiplatform.googleapis.com',
            'api_version': 'v1beta1',
        },
        'staging_bucket': f'gs://{PROJECT_ID}',
    }

    resource_name = get_agent_resource(agent_name)
    if not resource_name:
        # Deploy dummy agent to assign Agent Engine resource name
        remote_a2a_agent = client.agent_engines.create(
            agent=a2a_agent,
            config=config,
        )
    
    # Redeploy with Agent Engine resource name for VertexAiSessionService
    resource_name = get_agent_resource(agent_name)
    a2a_agent = A2aAgent(
        agent_card=await get_agent_card(agent),
        agent_executor_builder=get_agent_executor_class(agent, resource_name)
    )
    remote_a2a_agent = client.agent_engines.update(
        name=resource_name,
        agent=a2a_agent,
        config=config,
    )

/var/tmp/ipykernel_8184/1771281126.py:49: UserWarning: [EXPERIMENTAL] AgentCardBuilder: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  builder = AgentCardBuilder(agent=agent)


## Define RemoteA2aAgent instances and Root agent

In [3]:
def get_agent_resource(agent_name):
    for agent in agent_engines.list():
        if agent.display_name == agent_name:
            return agent.resource_name
    return None


class GoogleAuthRefresh(httpx.Auth):
    def __init__(self, scopes):
        self.credentials, _ = default(scopes=scopes)
        self.transport_request = Request()
        self.credentials.refresh(self.transport_request)

    def auth_flow(self, request):
        if not self.credentials.valid:
            self.credentials.refresh(self.transport_request)
        
        request.headers['Authorization'] = f'Bearer {self.credentials.token}'
        yield request


factory = ClientFactory(
    ClientConfig(
        supported_transports=[TransportProtocol.http_json],
        use_client_preference=True,
        httpx_client=httpx.AsyncClient(
            timeout=60,
            headers={
                'Content-Type': 'application/json',
            },
            auth=GoogleAuthRefresh(scopes=['https://www.googleapis.com/auth/cloud-platform']) 
        ),
    )
)

resource_name = get_agent_resource('research_agent1_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
research_agent1_remoteA2a = RemoteA2aAgent(
    name='research_agent1',
    description='''
An agent that gathers the necessary information for writing an article
and compiles it into a report (theme selection)
    ''',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('research_agent2_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
research_agent2_remoteA2a = RemoteA2aAgent(
    name='research_agent2',
    description='''
An agent that collects the information necessary for writing an article
and compiles it into a report (report creation)
    ''',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('writer_agent_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
writer_agent_remoteA2a = RemoteA2aAgent(
    name='writer_agent',
    description='An agent that writes feature articles on specific topics.',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('review_agent_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
review_agent_remoteA2a = RemoteA2aAgent(
    name='review_agent',
    description='An agent that reviews feature articles.',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

/var/tmp/ipykernel_71356/4262954498.py:38: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  research_agent1_remoteA2a = RemoteA2aAgent(
/var/tmp/ipykernel_71356/4262954498.py:50: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  research_agent2_remoteA2a = RemoteA2aAgent(
/var/tmp/ipykernel_71356/4262954498.py:62: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Imple

In [4]:
def get_print_agent(text):
    def before_model_callback(
        callback_context: CallbackContext, llm_request: LlmRequest
    ) -> LlmResponse:
        return LlmResponse(
            content=Content(
                role='model', parts=[Part(text=text)],
            )
        )
        
    return LlmAgent(
        name='print_agent',
        model='gemini-2.0-flash', # not used
        description='',
        instruction = '',
        before_model_callback=before_model_callback,
    )

research_agent = SequentialAgent(
    name='research_agent',
    sub_agents=[
        get_print_agent('\n---\n## The research agent will create a research report.\n---\n'),
        get_print_agent('\n## Selecting the topics for investigation.\n'),
        research_agent1_remoteA2a,
        get_print_agent('\n## Creating the research report based on the selected topics.\n'),
        research_agent2_remoteA2a,
        get_print_agent('\n#### The research report is ready. Shall we proceed with writing the article?\n'),
    ],
    description='An agent that collects the information necessary for writing an article and compiles it into a report.',
)

write_and_review_agent = SequentialAgent(
    name='write_and_review_agent',
    sub_agents=[
        get_print_agent('\n---\n## The writer agent will now write the article.\n---\n'),
        writer_agent_remoteA2a,
        get_print_agent('\n---\n## The review agent will now review the article.\n---\n'),
        review_agent_remoteA2a,
        get_print_agent('\n#### Would you like to request a revision of the article based on the review?\n'),
    ],
    description='Create and review an article.',
)

root_agent = LlmAgent(
    name='article_generation_flow',
    model='gemini-2.0-flash',
    instruction = '''
If asked what you can do, please respond with a clear and friendly summary of the following process:

- Execute a workflow to create an article on a theme specified by the user.
- First, create a research report on the theme.
- After that, the writer and review agents will collaborate to create an article that adheres to the editorial policy.

If the user specifies a theme for the article, execute the following flow:

1. Announce that you will begin creating an article on that theme and forward the task to research_agent to request a research report.
2. If the user confirms the creation of the article, forward the task to write_and_review_agent to request the article's creation and review.
3. If the user wishes to revise the article, forward the task to write_and_review_agent.

**Conditions**
The nickname for research_agent is "the Research Agent".
The nickname for write_and_review_agent is "the Writer and Review Agents".

## IMPORTANT ##
MAKE SURE THAT YOU ALWAYS CALL THE APPROPREATE SUBAGENT research_agent or write_and_review_agent WHEN NECESSARY.
''',
    sub_agents=[
        research_agent,
        write_and_review_agent,
    ],
    description='An agent that executes the workflow for creating an article.'
)

## Executon example

In [15]:
client = LocalApp(root_agent)

query = '''
Hello, what can you do?
'''
result = await client.stream(query)

I can execute a workflow to create an article on a theme specified by you. First, I'll ask the Research Agent to create a research report on the theme. After that, the Writer and Review Agents will collaborate to create an article that adheres to the editorial policy.



In [16]:
query = '''
Create an article with the theme "Spicy Arrangements of Local Cuisine
to Get Through the Summer."
'''
result = await client.stream(query)

/home/jupyter/.local/lib/python3.10/site-packages/google/adk/agents/remote_a2a_agent.py:382: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  converted_part = self._genai_part_converter(part)


Okay, I will begin creating an article on the theme "Spicy Arrangements of Local Cuisine to Get Through the Summer." I will now forward the task to the Research Agent to request a research report.


---
## The research agent will create a research report.
---


## Selecting the topics for investigation.



/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:210: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  return convert_a2a_message_to_event(
/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:280: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  par

Here is the research report outlining topics for the article "Spicy Arrangements of Local Cuisine to Get Through the Summer":

---

**Research Report: Spicy Arrangements of Local Cuisine to Get Through the Summer**

**Theme:** Spicy Arrangements of Local Cuisine to Get Through the Summer

**Objective:** To gather information for an article that explores how various local cuisines utilize spicy elements to cope with hot weather, offering both cultural insights and practical ideas.

**Proposed Topics for Investigation:**

1.  **The Cultural and Physiological Rationale Behind Summer Spiciness:**
    *   Explore why many cultures traditionally consume spicy food during hot summer months.
    *   Discuss the physiological effects (e.g., inducing sweat for natural cooling, stimulating appetite in heat, boosting metabolism).
    *   Highlight historical and cultural perspectives on this seemingly counter-intuitive practice.

2.  **Global Examples of Iconic Spicy Summer Local Dishes:**
    *  

In [17]:
query = '''
Yes, go ahead.
'''
result = await client.stream(query)

/home/jupyter/.local/lib/python3.10/site-packages/google/adk/agents/remote_a2a_agent.py:382: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  converted_part = self._genai_part_converter(part)


Okay, I will forward the task to the Writer and Review Agents to create the article.


---
## The writer agent will now write the article.
---



/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:210: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  return convert_a2a_message_to_event(
/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:280: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  par

When the summer sun beats down, reaching for a fiery dish might seem counterintuitive, but cultures worldwide have perfected the art of heating up to cool down. Get ready to discover why a little spice is just what you need to beat the heat!

## Overview

Ever wonder why so many hot countries crave spicy food when it's sweltering outside? It's not just about tradition; there's some cool science behind it! Eating spicy foods containing capsaicin (that's the stuff that makes chilies hot) tricks your body into thinking it's overheating. What does your body do? It sweats! As that sweat evaporates, it helps to naturally cool you down. Plus, when the heat zaps your appetite, a zesty, spicy dish can be just the ticket to get your taste buds going again. It's a clever trick humanity has perfected over centuries to stay refreshed and energized even on the warmest days.

## Global Spice Smarts

From Seoul to Bangkok, and all the way to Mexico City, local cuisines offer incredible examples of how

In [18]:
query = '''
Yes, go ahead.
'''
result = await client.stream(query)

/home/jupyter/.local/lib/python3.10/site-packages/google/adk/agents/remote_a2a_agent.py:382: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  converted_part = self._genai_part_converter(part)


Okay, I will forward the task to the Writer and Review Agents to revise the article based on the review.


---
## The writer agent will now write the article.
---



/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:210: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  return convert_a2a_message_to_event(
/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:280: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  par

# Beat Summer Heat: Eat Spicy Today!

When the summer sun beats down, what’s your first instinct? Probably a scoop of ice cream or a tall, cold drink, right? But what if I told you that turning up the heat in your kitchen could actually help you beat the heat outside? It sounds counterintuitive, but many cultures around the world have known this secret for centuries.

## ☀️ Overview

Eating spicy food might make you sweat, and that's precisely the point! When you consume capsaicin, the compound that gives chili peppers their kick, it triggers heat receptors in your mouth and throughout your body. This sends a signal to your brain that your body temperature is rising, even if it's not. Your body then responds by activating its natural cooling system: sweating. As that sweat evaporates from your skin, it takes heat with it, leaving you feeling cooler and more refreshed. It's like having your own internal air conditioner!

## 🌶️ Global Spice Smarts

You'll find this culinary wisdom woven 

In [19]:
query = f'''
Good job, show the entire article again.
'''
result = await client.stream(query)

# Beat Summer Heat: Eat Spicy Today!

As the summer sun relentlessly beats down today, what’s your first instinct? Probably a scoop of ice cream or a tall, cold drink, right? But what if I told you that turning up the heat in your kitchen could actually help you beat the heat outside? It sounds counterintuitive, but many cultures around the world have known this secret for centuries.

## ☀️ Overview

Eating spicy food might make you sweat, and that's precisely the point! When you consume capsaicin, the compound that gives chili peppers their kick, it triggers heat receptors in your mouth and throughout your body. This sends a signal to your brain that your body temperature is rising, even if it's not. Your body then responds by activating its natural cooling system: sweating. As that sweat evaporates from your skin, it takes heat with it, leaving you feeling cooler and more refreshed. It's like having your own internal air conditioner!

## 🌶️ Global Spice Smarts

You'll find this culin

## Deploying the root agent on Agent Engine

Execute the following command from Cloud Shell to assign `aiplatform.user` role to the Reasoning Engine service account so that the root agent can call remote agents.

```
PROJECT_ID=$(gcloud config list --format 'value(core.project)')
PROJECT_NUMBER=$(gcloud projects describe $PROJECT_ID --format='value(projectNumber)')
gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member="serviceAccount:service-${PROJECT_NUMBER}@gcp-sa-aiplatform-re.iam.gserviceaccount.com" \
    --role='roles/aiplatform.user'
```

Currently, deploying the root agent with `RemoteA2aAgent` as its subagents needs some workarounds to keep the root agent instance serializable.

We define a wrapper class for `ClientFactory` and `RemoteA2aAgent` as below.

In [ ]:
def get_agent_resource(agent_name):
    for agent in agent_engines.list():
        if agent.display_name == agent_name:
            return agent.resource_name
    return None


class GoogleAuthRefresh(httpx.Auth):
    def __init__(self, scopes):
        self.credentials, _ = default(scopes=scopes)
        self.transport_request = Request()
        self.credentials.refresh(self.transport_request)

    def auth_flow(self, request):
        if not self.credentials.valid:
            self.credentials.refresh(self.transport_request)        
        request.headers['Authorization'] = f'Bearer {self.credentials.token}'
        yield request


from a2a.client.transports.rest import RestTransport       
class MyClientFactory(ClientFactory):
    def create(self, card, consumers=None, interceptors=None):
        if not self._config.httpx_client:
            self._config.httpx_client=httpx.AsyncClient(
                timeout=60,
                headers={'Content-Type': 'application/json'},
                auth=GoogleAuthRefresh(scopes=['https://www.googleapis.com/auth/cloud-platform']) 
            )
            self._register_defaults(self._config.supported_transports)
        return super().create(card, consumers, interceptors)


class MyRemoteA2aAgent(RemoteA2aAgent):
    async def _ensure_httpx_client(self) -> httpx.AsyncClient:
        if not self._httpx_client:
            self._httpx_client=httpx.AsyncClient(
                timeout=60,
                headers={'Content-Type': 'application/json'},
                auth=GoogleAuthRefresh(scopes=['https://www.googleapis.com/auth/cloud-platform']) 
            )
        return self._httpx_client


factory = MyClientFactory(
    ClientConfig(
        supported_transports=[TransportProtocol.http_json],
        use_client_preference=True,
    )
)

Define RemoteA2aAgent instances with the wrapper classes, and the root agent.

In [4]:
resource_name = get_agent_resource('research_agent1_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
research_agent1_remoteA2a = MyRemoteA2aAgent(
    name='research_agent1',
    description='''
An agent that gathers the necessary information for writing an article
and compiles it into a report (theme selection)
    ''',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('research_agent2_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
research_agent2_remoteA2a = MyRemoteA2aAgent(
    name='research_agent2',
    description='''
An agent that collects the information necessary for writing an article
and compiles it into a report (report creation)
    ''',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('writer_agent_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
writer_agent_remoteA2a = MyRemoteA2aAgent(
    name='writer_agent',
    description='An agent that writes feature articles on specific topics.',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('review_agent_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
review_agent_remoteA2a = MyRemoteA2aAgent(
    name='review_agent',
    description='An agent that reviews feature articles.',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

def get_print_agent(text):
    def before_model_callback(
        callback_context: CallbackContext, llm_request: LlmRequest
    ) -> LlmResponse:
        return LlmResponse(
            content=Content(
                role='model', parts=[Part(text=text)],
            )
        )
        
    return LlmAgent(
        name='print_agent',
        model='gemini-2.0-flash', # not used
        description='',
        instruction = '',
        before_model_callback=before_model_callback,
    )

research_agent = SequentialAgent(
    name='research_agent',
    sub_agents=[
        get_print_agent('\n---\n## The research agent will create a research report.\n---\n'),
        get_print_agent('\n## Selecting the topics for investigation.\n'),
        research_agent1_remoteA2a,
        get_print_agent('\n## Creating the research report based on the selected topics.\n'),
        research_agent2_remoteA2a,
        get_print_agent('\n#### The research report is ready. Shall we proceed with writing the article?\n'),
    ],
    description='An agent that collects the information necessary for writing an article and compiles it into a report.',
)

write_and_review_agent = SequentialAgent(
    name='write_and_review_agent',
    sub_agents=[
        get_print_agent('\n---\n## The writer agent will now write the article.\n---\n'),
        writer_agent_remoteA2a,
        get_print_agent('\n---\n## The review agent will now review the article.\n---\n'),
        review_agent_remoteA2a,
        get_print_agent('\n#### Would you like to request a revision of the article based on the review?\n'),
    ],
    description='Create and review an article.',
)

root_agent = LlmAgent(
    name='article_generation_flow',
    model='gemini-2.0-flash',
    instruction = '''
If asked what you can do, please respond with a clear and friendly summary of the following process:

- Execute a workflow to create an article on a theme specified by the user.
- First, create a research report on the theme.
- After that, the writer and review agents will collaborate to create an article that adheres to the editorial policy.

If the user specifies a theme for the article, execute the following flow:

1. Announce that you will begin creating an article on that theme and forward the task to research_agent to request a research report.
2. If the user confirms the creation of the article, forward the task to write_and_review_agent to request the article's creation and review.
3. If the user wishes to revise the article, forward the task to write_and_review_agent.

**Conditions**
The nickname for research_agent is "the Research Agent".
The nickname for write_and_review_agent is "the Writer and Review Agents".

## IMPORTANT ##
MAKE SURE THAT YOU ALWAYS CALL THE APPROPREATE SUBAGENT research_agent or write_and_review_agent WHEN NECESSARY.

''',
    sub_agents=[
        research_agent,
        write_and_review_agent,
    ],
    description='An agent that executes the workflow for creating an article.'
)

Deploy the root agent on Agent Engine.

In [5]:
remote_agent = agent_engines.create(
    agent_engine=root_agent,
    display_name='article_generation_flow_agent',
    requirements=[
        'google-adk==1.14.1',
        'google-genai==1.36.0',
        'google-cloud-aiplatform==1.113.0',
        'a2a-sdk==0.3.5'
    ],
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'cloudpickle': '3.1.1', 'pydantic': '2.11.9', 'google-cloud-aiplatform': '1.113.0'}
The following requirements are missing: {'cloudpickle', 'pydantic'}
The following requirements are appended: {'pydantic==2.11.9', 'cloudpickle==3.1.1'}
The final list of requirements: ['google-adk==1.14.1', 'google-genai==1.36.0', 'google-cloud-aiplatform==1.113.0', 'a2a-sdk==0.3.5', 'pydantic==2.11.9', 'cloudpickle==3.1.1']
Using bucket etsuji-15pro-poc
Wrote to gs://etsuji-15pro-poc/agent_engine/agent_engine.pkl
Writing to gs://etsuji-15pro-poc/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://etsuji-15pro-poc/agent_engine/dependencies.tar.gz
failed to generate schema for async_add_session_to_memory: `async_add_session_to_memory` is not fully defined; you should define `Session`, then call `async_add_session_to_memory.model_rebuild()`.

For further information visit htt

Local application class to execute the root agent on Agent Engine.

In [6]:
class RemoteApp:
    def __init__(self, remote_agent, user_id='default_user'):
        self._remote_agent = remote_agent
        self._user_id = user_id
        self._session = None

    async def _stream(self, query):
        if not self._session:
            self._session = await remote_agent.async_create_session(user_id=self._user_id)
        events = self._remote_agent.async_stream_query(
            user_id=self._user_id,
            session_id=self._session['id'],
            message=query,
        )
        result = []
        async for event in events:
            if ('content' in event and 'parts' in event['content']):
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                print(response)
                result.append(response)
        return result

    async def stream(self, query):
        for _ in range(5):
            result = await self._stream(query)
            if result:
                return result
            time.sleep(3)
        return 'Error: No response from Agent Engine.'

Execute the root agent on Agent Engine with the `RemoteApp` instance.

In [7]:
client = RemoteApp(remote_agent)

query = '''
Hello, what can you do?
'''
result = await client.stream(query)

I can execute a workflow to create an article on a theme you specify. First, I'll create a research report on the theme using the Research Agent. After that, the Writer and Review Agents will collaborate to create an article that adheres to the editorial policy.



In [8]:
query = '''
Create an article with the theme "Spicy Arrangements of Local Cuisine
to Get Through the Summer."
'''
result = await client.stream(query)

Okay, I will begin creating an article on the theme "Spicy Arrangements of Local Cuisine to Get Through the Summer." I will now forward the task to the Research Agent to request a research report.



---
## The research agent will create a research report.
---


## Selecting the topics for investigation.

Here is a list of 5 topics for your research report on "Spicy Arrangements of Local Cuisine to Get Through the Summer":

1.  **The Paradoxical Cool: How Spicy Foods Combat Summer Heat**:
    *   Explore the physiological mechanisms (e.g., capsaicin stimulating thermoreceptors, inducing sweating, vasodilation) that contribute to a cooling sensation from spicy food.
    *   Discuss cultural beliefs and historical context across different regions (e.g., Ayurvedic principles, traditional Chinese medicine) that advocate for consuming piquant dishes in hot climates.
    *   Explain why certain "spicy arrangements" are specifically beneficial in summer, beyond just the heat.

2.  **Global He